# Week 7 — Part 03: Error handling that teaches the user what to do

**Estimated time:** 60–90 minutes

---

## Pre-study (Level 0)

Level 1 assumes Level 0 is complete. If you need a refresher on exceptions and debugging patterns:

- [Level 1 Pre-study index](../PRESTUDY.md)
- [Level 0 — Modules and exception handling](../../level_0/Chapters/2/02_modules_exceptions.md)

---

## What success looks like (end of Part 03)

- You can classify common failure types (input/config/external/output).
- You can turn an exception into a short user-facing message that suggests a next step.
- You validate inputs early (missing file / missing columns) with clear errors.

### Checkpoint

After running this notebook, you should be able to:

- run `classify_error(...)` on a few exceptions and see stable categories
- run `user_facing_message(...)` and get an actionable message

## Learning Objectives

- Write error messages that explain what went wrong and what to do next
- Classify common failure types (input, config, external, parsing)
- Validate inputs early with clear errors

## Overview

In this lab you will make failures explainable.

You will:

- validate inputs early (file exists, non-empty, required columns)
- classify exceptions into simple categories
- emit short, actionable user-facing messages

If you want the deeper discussion about errors as “product surface”, use the Level 0 links at the top of the notebook.

In [ ]:
from pathlib import Path
from typing import List


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError(
            "Input file not found: %s. Check the path or run --help for usage." % p
        )
    if p.stat().st_size == 0:
        raise ValueError("Input file is empty: %s. Provide a non-empty CSV." % p)
    return p


def require_columns(df, required: List[str]) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(
            "Missing required columns: %s. Update your schema or fix the input file." % missing
        )


print("error helpers ready")

In [ ]:
def classify_error(exc: Exception) -> str:
    # TODO: categorize errors into input/config/external/output types.
    if isinstance(exc, (FileNotFoundError, ValueError)):
        return "input"
    if isinstance(exc, RuntimeError):
        return "config"
    return "external"


def user_facing_message(exc: Exception) -> str:
    # TODO: map error type to a short, actionable message.
    kind = classify_error(exc)
    if kind == "input":
        return "Input error: %s" % str(exc)
    if kind == "config":
        return "Config error: %s" % str(exc)
    if kind == "output":
        return "Output error: %s" % str(exc)
    return "External error: %s" % str(exc)


print(user_facing_message(FileNotFoundError("missing.csv")))
print(user_facing_message(RuntimeError("Missing API_KEY")))
print("Implement classify_error() and user_facing_message().")

## Self-check

- If a beginner runs your project wrong, does the error message teach them what to do?

## References

- Python errors/exceptions: https://docs.python.org/3/tutorial/errors.html

## Appendix: Solutions (peek only after trying)

Reference implementations for `classify_error` and `user_facing_message`.

In [ ]:
def classify_error(exc: Exception) -> str:
    if isinstance(exc, FileNotFoundError):
        return "input"
    if isinstance(exc, ValueError):
        return "input"
    msg = str(exc).lower()
    if "api_key" in msg or "missing" in msg and "env" in msg:
        return "config"
    if "json" in msg or "schema" in msg or "parse" in msg:
        return "output"
    return "external"


def user_facing_message(exc: Exception) -> str:
    kind = classify_error(exc)
    if kind == "input":
        return "%s\nNext: check the input path/CSV schema." % str(exc)
    if kind == "config":
        return "%s\nNext: set required env vars (e.g. API_KEY) and retry." % str(exc)
    if kind == "output":
        return "%s\nNext: inspect the raw model output saved under output/." % str(exc)
    return "%s\nNext: retry with backoff or check provider status." % str(exc)


print("solution_message:", user_facing_message(ValueError("invalid JSON")))